COCO JSON → YOLO 학습용 형식 변환

1. images/train/*.png

2. labels/train/*.txt ← YOLO 라벨 (class x_center y_center width height)

3. dataset.yaml ← YOLO 학습 설정 파일

yolo format: Label ID, Center x, Center y, Width, Height

YOLO는 이미지 크기와 관계없이 학습할 수 있도록, 모든 값이 상대 좌표로 들어감
중심 좌표 기준이기 때문에 변환이 필요함

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jihyungpang","key":"2d5e377d246a0d9d8c94fe118214f71b"}'}

In [2]:
import os
import zipfile

# kaggle.json을 숨은 디렉토리에 복사
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# 권한 설정
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!pip install -q kaggle

In [4]:
!kaggle competitions download -c ai02-level1-project

 99% 3.88G/3.91G [00:18<00:00, 56.0MB/s]
100% 3.91G/3.91G [00:18<00:00, 221MB/s] 


In [5]:
!unzip -q ai02-level1-project.zip -d ./ai02_data

In [6]:
!pip install koreanize-matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 99.6 MB/s eta 0:00:00


In [7]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns
from tqdm import tqdm
import cv2

In [8]:
train_image_dir = './ai02_data/train_images'
test_image_dir = './ai02_data/test_images'
annotation_dir = './ai02_data/train_annotations'

In [9]:
# Load all annotation JSONs
all_images = []
all_annotations = []
all_categories = []

for root, _, files in os.walk(annotation_dir):
    for file in files:
        if file.endswith('.json'):
            with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                data = json.load(f)
                for img in data.get("images", []):
                    img["source_file"] = file
                all_images.extend(data.get("images", []))
                all_annotations.extend(data.get("annotations", []))
                all_categories.extend(data.get("categories", []))

df_images = pd.DataFrame(all_images)
df_annotations = pd.DataFrame(all_annotations)
df_categories = pd.DataFrame(all_categories).drop_duplicates("id")

In [10]:
pd.set_option('display.max_columns', None)

In [11]:
df_images.head()

,file_name,width,height,imgfile,drug_N,drug_S,back_color,drug_dir,light_color,camera_la,camera_lo,size,dl_idx,dl_mapping_code,dl_name,dl_name_en,img_key,dl_material,dl_material_en,dl_custom_shape,dl_company,dl_company_en,di_company_mf,di_company_mf_en,item_seq,di_item_permit_date,di_class_no,di_etc_otc_code,di_edi_code,chart,drug_shape,thick,leng_long,leng_short,print_front,print_back,color_class1,color_class2,line_front,line_back,img_regist_ts,form_code_name,mark_code_front_anal,mark_code_back_anal,mark_code_front_img,mark_code_back_img,mark_code_front,mark_code_back,change_date,id,source_file
0,K-003483-020238-027733-031885_0_2_0_2_75_000_2...,976,1280,K-003483-020238-027733-031885_0_2_0_2_75_000_2...,K-003483,정상알약,연회색 배경,앞면,주백색,75,0,200,3482,K-003483,기넥신에프정(은행엽엑스)(수출용),Ginexin-F Tab.,http://connectdi.com/design/img/drug/148609543...,은행엽건조엑스,Ginkgo Leaf Ext.,"정제, 저작정",에스케이케미칼(주),Sk Chemicals,,,199102014,19910906,[02190]기타의 순환계용약,일반의약품,"644700130,A18901361",진녹색의 타원형 필름 코팅정,장방형,3.9,11.1,6.1,SK,G40,초록,,,,20041222,필름코팅정,,,,,,,20200723,765,K-003483-020238-027733-031885_0_2_0_2_75_000_2...
1,K-003483-020238-027733-031885_0_2_0_2_90_000_2...,976,1280,K-003483-020238-027733-031885_0_2_0_2_90_000_2...,K-003483,정상알약,연회색 배경,앞면,주백색,90,0,200,3482,K-003483,기넥신에프정(은행엽엑스)(수출용),Ginexin-F Tab.,http://connectdi.com/design/img/drug/148609543...,은행엽건조엑스,Ginkgo Leaf Ext.,"정제, 저작정",에스케이케미칼(주),Sk Chemicals,,,199102014,19910906,[02190]기타의 순환계용약,일반의약품,"644700130,A18901361",진녹색의 타원형 필름 코팅정,장방형,3.9,11.1,6.1,SK,G40,초록,,,,20041222,필름코팅정,,,,,,,20200723,764,K-003483-020238-027733-031885_0_2_0_2_90_000_2...
2,K-003483-020238-027733-031885_0_2_0_2_70_000_2...,976,1280,K-003483-020238-027733-031885_0_2_0_2_70_000_2...,K-027733,정상알약,연회색 배경,앞면,주백색,70,0,200,27732,K-027733,트윈스타정 40/5mg,Twynsta Tab. 40/5mg,http://connectdi.com/design/img/drug/1MnfO1MVz...,텔미사르탄|암로디핀베실산염,Amlodipine Besylate| Telmisartan,"정제, 저작정",한국베링거인겔하임(주),Boehringer Ingelheim Korea,,,201005081,20100819,[02140]혈압강하제,전문의약품,653501270,흰색과 연한 파란색의 양면이 볼록한 타원형 이중정이다.,타원형,5.0,14.0,6.8,마크 A1,,하양,파랑,,,20101027,다층정,"집,화살,교회",,http://connectdi.com/design/img/drug/147938647...,,,,20200304,763,K-003483-020238-027733-031885_0_2_0_2_70_000_2...
3,K-003483-020238-027733-031885_0_2_0_2_75_000_2...,976,1280,K-003483-020238-027733-031885_0_2_0_2_75_000_2...,K-027733,정상알약,연회색 배경,앞면,주백색,75,0,200,27732,K-027733,트윈스타정 40/5mg,Twynsta Tab. 40/5mg,http://connectdi.com/design/img/drug/1MnfO1MVz...,텔미사르탄|암로디핀베실산염,Amlodipine Besylate| Telmisartan,"정제, 저작정",한국베링거인겔하임(주),Boehringer Ingelheim Korea,,,201005081,20100819,[02140]혈압강하제,전문의약품,653501270,흰색과 연한 파란색의 양면이 볼록한 타원형 이중정이다.,타원형,5.0,14.0,6.8,마크 A1,,하양,파랑,,,20101027,다층정,"집,화살,교회",,http://connectdi.com/design/img/drug/147938647...,,,,20200304,765,K-003483-020238-027733-031885_0_2_0_2_75_000_2...
4,K-003483-020238-027733-031885_0_2_0_2_90_000_2...,976,1280,K-003483-020238-027733-031885_0_2_0_2_90_000_2...,K-027733,정상알약,연회색 배경,앞면,주백색,90,0,200,27732,K-027733,트윈스타정 40/5mg,Twynsta Tab. 40/5mg,http://connectdi.com/design/img/drug/1MnfO1MVz...,텔미사르탄|암로디핀베실산염,Amlodipine Besylate| Telmisartan,"정제, 저작정",한국베링거인겔하임(주),Boehringer Ingelheim Korea,,,201005081,20100819,[02140]혈압강하제,전문의약품,653501270,흰색과 연한 파란색의 양면이 볼록한 타원형 이중정이다.,타원형,5.0,14.0,6.8,마크 A1,,하양,파랑,,,20101027,다층정,"집,화살,교회",,http://connectdi.com/design/img/drug/147938647...,,,,20200304,764,K-003483-020238-027733-031885_0_2_0_2_90_000_2...


In [12]:
df_annotations.head()

,area,iscrowd,bbox,category_id,ignore,segmentation,id,image_id
0,57684,0,"[579, 759, 276, 209]",3482,0,[],2899,765
1,57330,0,"[609, 791, 273, 210]",3482,0,[],2900,764
2,80367,0,"[566, 261, 301, 267]",27732,0,[],2906,763
3,91409,0,"[86, 727, 323, 283]",27732,0,[],2904,765
4,86900,0,"[122, 767, 316, 275]",27732,0,[],2905,764


In [13]:
df_annotations.shape

(4526, 8)

In [14]:
df_categories.head()

,supercategory,id,name
0,pill,3482,기넥신에프정(은행엽엑스)(수출용)
2,pill,27732,트윈스타정 40/5mg
5,pill,31884,자누메트엑스알서방정 100/1000mg
7,pill,20237,플라빅스정 75mg
10,pill,18356,종근당글리아티린연질캡슐(콜린알포세레이트)


In [15]:
unique_categories = sorted(df_categories["id"].unique())
category_id_to_class_id = {cat_id: idx for idx, cat_id in enumerate(unique_categories)}

In [ ]:
import os

label_dir = "./labels"
os.makedirs(label_dir, exist_ok=True)  # 디렉토리 없으면 생성

for image_id in df_images["id"].unique():
    image_info = df_images[df_images["id"] == image_id].iloc[0]
    annotations = df_annotations[df_annotations["image_id"] == image_id]

    label_lines = []
    for _, ann in annotations.iterrows():
        x, y, w, h = ann["bbox"]
        img_w, img_h = image_info["width"], image_info["height"]

        # 중심좌표로 변환 + 정규화
        x_center = (x + w / 2) / img_w
        y_center = (y + h / 2) / img_h
        w_norm = w / img_w
        h_norm = h / img_h

        class_id = ann["category_id"]  # class_id는 나중에 category 이름과 매핑 필요

        line = f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}"
        label_lines.append(line)

    # 이미지 이름에서 확장자 제거
    image_filename = os.path.splitext(image_info["file_name"])[0]
    label_path = os.path.join(label_dir, image_filename + ".txt")

    with open(label_path, "w") as f:
        f.write("\n".join(label_lines))

안녕

In [16]:
# 새로
label_dir = "./labels"
os.makedirs(label_dir, exist_ok=True)

for image_id in df_images["id"].unique():
    image_info = df_images[df_images["id"] == image_id].iloc[0]
    annotations = df_annotations[df_annotations["image_id"] == image_id]

    label_lines = []
    for _, ann in annotations.iterrows():
        x, y, w, h = ann["bbox"]
        img_w, img_h = image_info["width"], image_info["height"]

        x_center = (x + w / 2) / img_w
        y_center = (y + h / 2) / img_h
        w_norm = w / img_w
        h_norm = h / img_h

        class_id = category_id_to_class_id[ann["category_id"]]
        line = f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}"
        label_lines.append(line)

    image_filename = os.path.splitext(image_info["file_name"])[0]
    label_path = os.path.join(label_dir, image_filename + ".txt")

    with open(label_path, "w") as f:
        f.write("\n".join(label_lines))

In [17]:
import shutil
from google.colab import files

# 라벨 디렉토리를 zip으로 압축
shutil.make_archive("labels_yolo", "zip", label_dir)

# 압축된 zip 파일 다운로드
files.download("labels_yolo.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
import os
import shutil
import random
from glob import glob

# 이미지 디렉토리 및 라벨 디렉토리 경로
image_dir = train_image_dir  # 예: "/content/ai02_data/train_images"
label_dir = "./labels"       # 예: "/content/ai02_data/labels/train"
#test_image_dir
# 결과 저장 디렉토리 설정
output_base = "./yolo_dataset"
img_train_dir = os.path.join(output_base, "images", "train")
img_val_dir = os.path.join(output_base, "images", "val")

lbl_train_dir = os.path.join(output_base, "labels", "train")
lbl_val_dir = os.path.join(output_base, "labels", "val")

# 디렉토리 생성
for d in [img_train_dir, img_val_dir, lbl_train_dir, lbl_val_dir]:
    os.makedirs(d, exist_ok=True)

# 이미지 파일 경로 리스트 만들기
image_paths = sorted(glob(os.path.join(image_dir, '*.png')))
random.seed(71)  # 재현 가능성 확보
random.shuffle(image_paths)

# 8:2로 나누기
val_count = int(len(image_paths) * 0.2)
val_paths = image_paths[:val_count]
train_paths = image_paths[val_count:]

# 복사 함수 정의
def copy_files(path_list, src_lbl_dir, dst_img_dir, dst_lbl_dir):
    for img_path in path_list:
        file_name = os.path.basename(img_path)
        base_name = os.path.splitext(file_name)[0]

        # 이미지 복사
        shutil.copy2(img_path, os.path.join(dst_img_dir, file_name))

        # 라벨 경로
        src_lbl_path = os.path.join(src_lbl_dir, base_name + '.txt')
        dst_lbl_path = os.path.join(dst_lbl_dir, base_name + '.txt')

        # 라벨 파일이 존재할 경우에만 복사
        if os.path.exists(src_lbl_path):
            shutil.copy2(src_lbl_path, dst_lbl_path)

# 복사 실행
copy_files(train_paths, label_dir, img_train_dir, lbl_train_dir)
copy_files(val_paths, label_dir, img_val_dir, lbl_val_dir)

print(f"✅ Train 이미지 수: {len(train_paths)}")
print(f"✅ Val 이미지 수: {len(val_paths)}")

✅ Train 이미지 수: 1192
✅ Val 이미지 수: 297


In [19]:
import os

output_base = "./yolo_dataset"
yaml_path = os.path.join(output_base, "dataset.yaml")

# df_categories에서 id 순서대로 정렬 후 클래스명 리스트 생성
df_categories_sorted = df_categories.sort_values("id")
class_names = df_categories_sorted["name"].tolist()
num_classes = len(class_names)

yaml_content = f"""
train: {os.path.join(output_base, "images/train")}
val: {os.path.join(output_base, "images/val")}

nc: {num_classes}
names: {class_names}
"""

# output_base 폴더가 없으면 생성
os.makedirs(output_base, exist_ok=True)

with open(yaml_path, "w") as f:
    f.write(yaml_content.strip())

print(f"YAML 파일이 생성되었습니다: {yaml_path}")

YAML 파일이 생성되었습니다: ./yolo_dataset/dataset.yaml


In [20]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [21]:
from google.colab import files
# output_base 폴더를 yolo_dataset.zip으로 압축
shutil.make_archive("yolo_dataset", 'zip', output_base)

files.download("yolo_dataset.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [23]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [24]:
model = YOLO("yolov8n.pt")
model.info()

100%|██████████| 6.25M/6.25M [00:00<00:00, 300MB/s]


YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(129, 3157200, 0, 8.8575488)

In [25]:
import torch
print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("사용 중인 디바이스:", torch.cuda.get_device_name(0))

CUDA 사용 가능 여부: True
사용 중인 디바이스: NVIDIA L4


In [28]:
results = model.train(
    data='/content/yolo_dataset/dataset.yaml',  # 너가 만든 YAML 경로로 수정
    epochs=100,
    imgsz=640,
    batch=16,  # GPU 메모리에 따라 조정
    device=0,
    cache=True
)

Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretr

100%|██████████| 22.2M/22.2M [00:00<00:00, 341MB/s]

Overriding model.yaml nc=80 with nc=73

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

Model summary: 129 layers, 3,096,741 parameters, 3,096,725 gradients, 8.6 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 382MB/s]


...

    100/100      4.46G     0.1865     0.6136     0.7926         19        640: 100%|██████████| 75/75 [00:07<00:00,  9.85it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:01<00:00,  7.25it/s]

                   all        297        910      0.785      0.966      0.869      0.862



100 epochs completed in 0.259 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.4MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.4MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 3,091,487 parameters, 0 gradients, 8.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  4.29it/s]


                   all        297        910      0.783       0.98      0.875      0.868
            보령부스파정 5mg         41         41      0.831          1      0.873      0.869
           뮤테란캡슐 100mg         40         40      0.813      0.977      0.873      0.862
           일양하이트린정 2mg         44         44      0.809          1       0.86      0.854
    기넥신에프정(은행엽엑스)(수출용)         95         95      0.823          1       0.83      0.823
     무코스타정(레바미피드)(비매품)         23         23      0.766          1      0.819      0.819
                  알드린정         10         10      0.575          1      0.808      0.808
          뉴로메드정(옥시라세탐)          2          2      0.788          1      0.995      0.945
            타이레놀정500mg          8          8      0.864          1      0.939      0.908
          에어탈정(아세클로페낙)          9          9      0.675          1       0.83       0.83
         삼남건조수산화알루미늄겔정          9          9      0.797          1      0.875      0.875
타이레놀이알서방정(아세트아미노펜)(수출

In [31]:
train_results = model.predict(
    source="/content/yolo_dataset/images/train",     # 테스트 이미지 디렉토리
    conf=0.25,                  # confidence threshold
    imgsz=640,                  # 이미지 크기
    save=True,                  # 예측 결과 이미지 저장
    save_txt=True,              # 바운딩 박스 txt 저장
    show_labels=True,           # 이미지에 라벨 표시
    show_conf=True,             # confidence score 표시
    name="predict"              # 저장 디렉토리 이름 (runs/detect/predict)
)


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1192 /content/yolo_dataset/images/train/K-001900-010224-016551-031705_0_2_0_2_70_000_200.png: 640x512 1 보령부스파정 5mg, 1 동아가바펜틴정 800mg, 1 낙소졸정 500/20mg, 68.1ms
image 2/1192 /content/yolo_dataset/images/train/K-001900-010224-016551-031705_0_2_0_2_90_000_200.png: 640x512 1 보령부스파정 5mg, 1 동아가바펜틴정 800mg, 1 낙소졸정 500/20mg, 8.7ms
image 3/1192 /content/yolo_dataset/images/train/K-001900-010224-016551-033009_0_2_0_2_70_000_200.png: 640x512 1 보령부스파정 5mg, 1 동

In [30]:
model = YOLO("runs/detect/train2/weights/best.pt")
metrics = model.val()
print(metrics.box)

Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 3,091,487 parameters, 0 gradients, 8.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4580.2±2214.2 MB/s, size: 1731.4 KB)


val: Scanning /content/yolo_dataset/labels/val.cache... 297 images, 0 backgrounds, 0 corrupt: 100%|██████████| 297/297 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:03<00:00,  4.92it/s]


                   all        297        910      0.783       0.98      0.875      0.868
            보령부스파정 5mg         41         41      0.831          1      0.874       0.87
           뮤테란캡슐 100mg         40         40      0.813      0.978      0.875      0.864
           일양하이트린정 2mg         44         44      0.809          1       0.86      0.854
    기넥신에프정(은행엽엑스)(수출용)         95         95      0.823          1       0.83      0.823
     무코스타정(레바미피드)(비매품)         23         23      0.766          1      0.819      0.819
                  알드린정         10         10      0.575          1      0.808      0.808
          뉴로메드정(옥시라세탐)          2          2      0.788          1      0.995      0.945
            타이레놀정500mg          8          8      0.864          1      0.939      0.908
          에어탈정(아세클로페낙)          9          9      0.675          1       0.83       0.83
         삼남건조수산화알루미늄겔정          9          9      0.797          1      0.875      0.875
타이레놀이알서방정(아세트아미노펜)(수출

In [32]:
import os
import random
from PIL import Image
import matplotlib.pyplot as plt

# 예측 결과 이미지가 저장된 디렉토리
predict_dir = "runs/detect/predict"

# 예측된 모든 이미지 파일 불러오기
all_images = [f for f in os.listdir(predict_dir) if f.lower().endswith((".jpg", ".png"))]

# 20개 이미지 무작위 선택
sample_images = random.sample(all_images, min(20, len(all_images)))

# 시각화 (10행 2열 = 한 줄에 2개씩 20개)
plt.figure(figsize=(10, 40))  # 너비=10, 높이=40으로 크게 설정

for i, img_file in enumerate(sample_images):
    img_path = os.path.join(predict_dir, img_file)
    img = Image.open(img_path)

    plt.subplot(10, 2, i + 1)
    plt.imshow(img)
    plt.title(img_file)
    plt.axis("off")

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [36]:
print(len(train_results))  # 예측된 이미지 수
print(train_results[0].boxes)  # 첫 번째 이미지의 바운딩 박스 정보

1192
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([23., 59.,  0.], device='cuda:0')
conf: tensor([0.9559, 0.8276, 0.7956], device='cuda:0')
data: tensor([[5.6936e+02, 1.2457e+02, 8.4997e+02, 5.8350e+02, 9.5594e-01, 2.3000e+01],
        [1.3552e+02, 1.2700e+02, 3.7965e+02, 5.4684e+02, 8.2758e-01, 5.9000e+01],
        [6.4036e+02, 8.6191e+02, 8.5509e+02, 1.0169e+03, 7.9559e-01, 0.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (1280, 976)
shape: torch.Size([3, 6])
xywh: tensor([[709.6660, 354.0344, 280.6042, 458.9298],
        [257.5858, 336.9183, 244.1323, 419.8446],
        [747.7205, 939.4113, 214.7300, 154.9973]], device='cuda:0')
xywhn: tensor([[0.7271, 0.2766, 0.2875, 0.3585],
        [0.2639, 0.2632, 0.2501, 0.3280],
        [0.7661, 0.7339, 0.2200, 0.1211]], device='cuda:0')
xyxy: tensor([[ 569.3638,  124.5695,  849.9681,  583.4993],
        [ 135.5196,  126.9960,  379.6519,  546.8406],
        [ 640.3555,  861.9127,  855.0854, 1016.9100

In [44]:
import os
import shutil
from glob import glob

# 원본 테스트 이미지 디렉토리
#test_image_dir = './ai02_data/test_images'

# YOLO 포맷 저장 경로
#output_base = './yolo_dataset'
img_test_dir = os.path.join(output_base, 'images', 'test')

if os.path.isfile(img_test_dir):
    os.remove(img_test_dir)

# 디렉토리 생성
os.makedirs(img_test_dir, exist_ok=True)

# 이미지 경로 가져오기
test_image_paths = sorted(glob(os.path.join(test_image_dir, '*.png')))

# 복사
for img_path in test_image_paths:
    file_name = os.path.basename(img_path)
    shutil.copy2(img_path, os.path.join(img_test_dir, file_name))

print(f"✅ Test 이미지 {len(test_image_paths)}개가 '{img_test_dir}'에 복사되었습니다.")

✅ Test 이미지 843개가 './yolo_dataset/images/test'에 복사되었습니다.


In [45]:
test_results = model.predict(
    source="/content/yolo_dataset/images/test",  # ✅ test 경로로 수정
    conf=0.05,
    imgsz=640,
    save=False,
    save_txt=False,
    name="predict_kaggle"
)


image 1/843 /content/yolo_dataset/images/test/1.png: 640x512 1 보령부스파정 5mg, 1 동아가바펜틴정 800mg, 1 놀텍정 10mg, 1 울트라셋이알서방정, 9.9ms
image 2/843 /content/yolo_dataset/images/test/10.png: 640x512 1 보령부스파정 5mg, 1 가바토파정 100mg, 1 라비에트정 20mg, 1 레일라정, 9.3ms
image 3/843 /content/yolo_dataset/images/test/100.png: 640x512 1 보령부스파정 5mg, 1 가바토파정 100mg, 1 란스톤엘에프디티정 30mg, 1 신바로정, 7.5ms
image 4/843 /content/yolo_dataset/images/test/1003.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 리피토정 20mg, 1 트윈스타정 40/5mg, 1 제미메트서방정 50/1000mg, 7.5ms
image 5/843 /content/yolo_dataset/images/test/1004.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 리피토정 20mg, 1 트윈스타정 40/5mg, 1 제미메트서방정 50/1000mg, 7.6ms
image 6/843 /content/yolo_dataset/images/test/1005.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 리피토정 20mg, 1 트윈스타정 40/5mg, 1 제미메트서방정 50/1000mg, 7.7ms
image 7/843 /content/yolo_dataset/images/test/1006.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 플라빅스정 75mg, 1 자누메트정 50/850mg, 1 아모잘탄정 5/100mg, 8.0ms
image 8/843 /content/yolo_dataset/images/test/1007.png: 640x512 1 

In [46]:
# 결과 파싱 및 저장
rows = []
annotation_id = 1

for image_idx, result in enumerate(test_results):
    image_id = image_idx + 1  # 또는 result.path에서 추출 가능

    boxes = result.boxes
    if boxes is None or boxes.xywh is None:
        continue

    for i in range(len(boxes.xywh)):
        x, y, w, h = boxes.xywh[i].tolist()
        score = float(boxes.conf[i])
        category_id = int(boxes.cls[i])

        # COCO-style: xmin, ymin 기준으로 좌표 변환
        xmin = round(x - w / 2)
        ymin = round(y - h / 2)
        bbox_w = round(w)
        bbox_h = round(h)

        rows.append([
            annotation_id,
            image_id,
            category_id,
            xmin,
            ymin,
            bbox_w,
            bbox_h,
            round(score, 2)
        ])
        annotation_id += 1

# CSV로 저장
df = pd.DataFrame(rows, columns=[
    "annotation_id", "image_id", "category_id",
    "bbox_x", "bbox_y", "bbox_w", "bbox_h", "score"
])

csv_path = "/content/submission.csv"
df.to_csv(csv_path, index=False)
print(f"✅ CSV 저장 완료: {csv_path} / 총 {len(df)}개의 바운딩 박스")

✅ CSV 저장 완료: /content/submission.csv / 총 3259개의 바운딩 박스


In [47]:
from google.colab import files
files.download(csv_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>